# Domácí úkol


In [2]:
# ===== Import necessary modules =====
import numpy as np
import matplotlib.pyplot as plt
import math
# Necessary for conversions from float to frac due to imperfect
# binary representation of decimal numbers (such as 2.1)
from fractions import Fraction as frac
from sympy import primerange

In [3]:
# ===== Use LaTeX ======
plt.rcParams['text.usetex'] = True
plt.rc('font', family='serif', size=12)
plt.rc('axes', titlesize=19)

## 1. Spinograf
- **Úkol : nakreslete obrázek trajektorie bodu na spinografu pro zadané hodnoty**
    - $R$ (poloměr větší kružnice),
    - $r$ (poloměr menší kružnice) a
    - $d$ (vzdálenost bodu od středu menší kružnice).
- Rozmyslete si, kolik rotací musí menší kružnice udělat, aby se bod vrátil do své původní pozice. (vykreslete dráhu tak, aby byla uzavřená)
- Vyzkoušejte alespoň pro $R=10, r=3, d=1$ a $R=7, r=2, d=5$.
- Vykreslete pomocí tenké modré čáry, zvolte dostatek bodů, aby byla trajektorie plynulá (hladká).

---

### Co je to spinograf?
Jedná se o hru/výukovou pomůcku (viz https://en.wikipedia.org/wiki/Spirograph).

Pro vysvětlení trajektorie bodu na spinografu (často označovaného jako hypotrochoida; když se menší kružnice otáčí uvnitř větší kružnice) použijeme goniometrický popis polohy bodu na kružnici v závislosti na úhlu rotace. Máme jednu větší kružnici s poloměrem $R$ a jednu menší kružnici s poloměrem $r$ a chceme trasovat dráhu bodu, který je od středu menší kružnice vzdálen $d$.


### Poloha středu menší kružnice

Střed menší kružnice se otáčí uvnitř větší kružnice. Poloha středu menší kružnice v závislosti na úhlu $\theta$ je dána parametrickými rovnicemi v kartézských souřadnicích:

-  $ X = (R - r)  \cos(\theta) $ 
-  $ Y = (R - r)  \sin(\theta) $ 

kde $\theta$ je úhel rotace středu menší kružnice v radiánech.

### Otočení menší kružnice

Menší kružnice se otočí (valí se po větší kružnici). Úhel, o který se menší kružnice otočila kolem svého středu vzhledem k počátečnímu stavu, je dán délkou oblouku, který urazila:
$$ \theta R $$

Pokud chceme tento oblouk vyjádřit v radiánech vzhledem k úhlu otočení, o který se otočila menší kružnice kolem svého středu, převádíme zpětně z oblouku na úhel:
$$ \theta \frac{R}{r} $$
Poslední dvě důležitá pozotování jsou:
- malá kružnice se otočí negativně (tedy úhel je záporný)
- malá kružnice se otočí zároveň se svým středem, tedy k úhlu otočení musíme přičíst úhel $\theta$

Celkově máme:

$$ \phi = \theta - \theta \frac{R}{r} = - \theta \frac{R-r}{r} $$


### Pozice bodu na spinografu

Nakonec sloučíme pozici středu a relativní pozici bodu ve vzdálenosti $d$ od středu menší kružnice:

-  $x = X + d \cos(\phi) = (R - r) \cos(\theta) + d \cos(- \theta \frac{R-r}{r})$ 
-  $y = Y + d \sin(\phi) = (R - r) \sin(\theta) + d \sin(- \theta \frac{R-r}{r})$ 

Pokud použijeme vlastnosti (sudost/lichost) gonimetricých funkcí, můžeme tyto rovnice upravit na:

$$ x = (R - r) \cos(\theta) + d \cos(\theta \frac{R-r}{r}) $$
$$ y = (R - r) \sin(\theta) - d \sin(\theta \frac{R-r}{r}) $$

In [106]:
# Draw a hypotrochoid
def hypotrochoid(R=10, r=7, d=1):
    # ==== RANGE AND FINENESS OF PARAMETER Θ ====
    # Find the necessary angle θ_max to return to close the curve:
    # I base the algorithm for finding the θ_max on following assumptions:
    # - Let O be the perimeter of the greater circle G and o that of the smaller circle S and let ν be
    # the radius vector of the center of the circle S being positioned in the center of the circle G
    # and θ the angle of ν.
    # Then the hypotrochoid is closed (first time) if θ is such that $ n * O = m * o $
    # where n,m are natural numbers and $ θ = 2πn $ (n,m being the smallest possible).
    # The number n actually represents the number of revolutions of the radius vector ν (which is determined by θ).
    # - All numbers in a computer are rational, indeed - so irrational (for which a solution does not always exist)
    # are excluded. Then the smallest solution of the equation $ n * O = m * o $ can be found this way:
    #   - Find a fractional form of O and o -> $ O = O_n/O_d $ and $ o = o_n/o_d $ respectively
    #   - Find the least common denominator (LCD) of those fractions (i.e. $lcd = lcm(O_d, o_d)$)
    #   - Find the numerators (O_N, o_N) respective to the LCD
    #   - Find their least common multiple (LCM)
    #   - Then $ n = LCM / O_n $
    # - To find the value of n with perimeters O and o, although more reasonable and clearer is this way, it is
    # computationally inefficient and unnecessary as O and o differ from known values of radii R and r respectively
    # by a factor 2π, and so the value of n can be calculated directly using the radii R and r instead of
    # the perimeters O and o.
    # - Knowing n, we can simply calculate θ_max from $ θ = 2πn $ (stated above)
    R_frac = frac(R).limit_denominator(1000)                # Fractional representation of R (within the precision)
    r_frac = frac(r).limit_denominator(1000)                # Fractional representation of r (within the precision)
    lcd = math.lcm(R_frac.denominator, r_frac.denominator)  # LCD of R and r
    R_numer = int(R * lcd)                                  # Numerator for R with denominator lcd
    r_numer = int(r * lcd)                                  # Numerator for r with denominator lcd
    numers_lcm = math.lcm(R_numer, r_numer)                 # LCM of numerators respective to lcd
    revols_of_G = numers_lcm / R_numer                      # Revolutions of the greater circle G to close the curve
    θ_min = 0                            # Angle of the radius vector ν at the "starting point" of the curve
    θ_max = 2 * np.pi * revols_of_G      # The least angle θ necessary to close the curve
    θ_fineness = int(100 * revols_of_G)  # Fineness set to 100 points for each revolution

    # ==== DATA GENERATION ====
    θ = np.linspace(θ_min, θ_max, θ_fineness)  # Vector of values of θ
    λ = R - r  # Temporary value (distance between centers of circles)
    κ = λ / r  # Temporary value (koefficient of θ)
    φ = κ * θ  # Temporary variable for the second goniometrical function
    x = λ * np.cos(θ) + d * np.cos(φ)  # x-coordinate values for parameter θ
    y = λ * np.sin(θ) - d * np.sin(φ)  # y-coordinate values for parameter θ

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(8, 8), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.plot(x, y, color="blue", linewidth=1, linestyle='-')
    # Set title
    axes.set_title(f"Spirograph (hypotrochoid) for ${R = }, {r = }, {d = }$")
    # Show the figure
    plt.show()

In [ ]:
# Tests
hypotrochoid(R=10, r=3, d=1)
hypotrochoid(R=7, r=2, d=5)
hypotrochoid()
hypotrochoid(R=20, r=7, d=2)
hypotrochoid(R=10, r=7, d=6)
hypotrochoid(R=10, r=7, d=8)
hypotrochoid(R=2.5, r=2, d=1)
hypotrochoid(R=2.1, r=2, d=1)   # This would not work if the fractions module were not used
hypotrochoid(R=2.11, r=2, d=1)  # This would not work if the fractions module were not used

**NOTE:** \
I was forced to use class `Fraction` from module `fractions` to find fractional representation of floats instead of using simpler `float.as_integer_ratio()` function. The reason is this: \
Some "simple" floats (such as 2.1, 0.1, ...) which have simple fractional representations are saved approximately as the closest binary floats, so 2.1 is not virtually 2.1 but something like 2.1000000...002. This causes that `(2.1).as_integer_ratio()` does not return the tuple `(21, 10)` but `(4_728_779_608_739_021, 2_251_799_813_685_248)`. Upon which account, the respective ***hypotrochoid closes after myriads of revolutions*** (4_503_599_627_370_496, i.e. quadrillions); furthermore, it accordingly requires about 3.12 EiB of the memory. \
The essential problem is a wrong curve (it does not correspond with the curve for 2.1). The other problem is that no such memory is available, of course. That is why I had to use `Fraction` which provides a method (`Fraction.limit_denominator()`) for setting denominator's maximal value. \
 \
This approach limits floats to have the precision only up to 1/1000. However, it can be tuned for a specific use-case.


# 2. Prvočíselná spirála


**Část 1.: Vykreslení "slunečnicových bodů"**

- Vykreslete tzv. "slunečnicové body". Tedy prvních `N` takových bodů.
- Vykreslete je tak, aby byl vzor dobře vidět (zvolte vhodné velikosti bodů)

##### Co jsou to slunečnicové body?
Poloměr $r$ a úhel $\theta$ pro každý bod (body číslovány přirozenými čísly: $n=1,...,N$) jsou určeny následovně:

1. Zlatý úhel $\phi$ je dán vztahem:
$$
\phi = \pi \cdot (3 - \sqrt{5})
$$

2. Poloměr $r$ je odvozen z indexu bodu $n$ takto:
$$
r = \sqrt{n}
$$

3. Úhel $\theta$ pro bod $n$ je vypočítán takto:
$$
\theta = n \cdot \phi
$$

Každý bod je poté umístěn do kartézských souřadnic $(x, y)$, které jsou vypočítány dle těchto vztahů:
$$
x = r \cdot \cos(\theta)
$$
$$
y = r \cdot \sin(\theta)
$$

In [115]:
# Draw a sunflower seed distribution
def sunflower_seed_distribution(N=233):
    # ==== DATA GENERATION ====
    φ = np.pi * (3 - math.sqrt(5))  # Golden angle
    # Polar coordinates
    n = np.arange(1, N + 1, 1)
    r = np.sqrt(n)
    θ = n * φ
    # Cartesian coordinates
    x = r * np.cos(θ)
    y = r * np.sin(θ)

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(8, 8), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.scatter(x, y, 
                 color="black", 
                 marker="o", 
                 s=(144 / N) * 100 * np.sqrt(n),  # Adjust sizes of markers
                 edgecolors="lightgrey")
    # Set title
    axes.set_title(f"Sunflower Seed Distribution for ${N = }$")
    # Show the figure
    plt.show()

In [ ]:
# Tests
sunflower_seed_distribution(144)
sunflower_seed_distribution()
sunflower_seed_distribution(377)
sunflower_seed_distribution(610)


**Část 2.: Vykreslení prvočíselné spirály**
- Vykreslete prvočíselnou spirálu pro prvních `N` prvočísel.
- Jedná se o stejný vzor jako v případě "slunečnicových bodů", ale tentokrát vykreslíme pouze ty body, které odpovídají prvočíslům.


In [51]:
# Draw a prime-number sunflower seed distribution
def primes_sunflower_seed_distribution(N=10_946):
    # ==== DATA GENERATION ====
    φ = np.pi * (3 - math.sqrt(5))  # Golden angle
    # Polar coordinates
    n = np.array([i for i in primerange(N + 1)])
    r = np.sqrt(n)
    θ = n * φ
    # Cartesian coordinates
    x = r * np.cos(θ)
    y = r * np.sin(θ)

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(8, 8), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.scatter(x, y,
                 color="black",
                 marker="o",
                 s=(144 / N) * 100 * np.sqrt(n),  # Adjust sizes of markers
                 edgecolors="lightgrey")
    # Set title
    axes.set_title(f"Prime-number Sunflower Seed Distribution for ${N = }$")
    # Show the figure
    plt.show()


In [ ]:
# Tests
primes_sunflower_seed_distribution(1597)
primes_sunflower_seed_distribution(2584)
primes_sunflower_seed_distribution(4181)
primes_sunflower_seed_distribution()
primes_sunflower_seed_distribution(17_711)
primes_sunflower_seed_distribution(28_657)
primes_sunflower_seed_distribution(46_368)
primes_sunflower_seed_distribution(75_025)
primes_sunflower_seed_distribution(121_393)
primes_sunflower_seed_distribution(1_346_269)

# Bonusové úkoly
1. Modifikujte spinograf tak, aby:
    - menší kružnice se otáčela po vnější straně větší kružnice
    - menší kružnice se otáčela po rovnostraném n-úhelníku (trojúhelníku, čtverci, šestiúhelníku, ...)

### Epitrochoid

In [67]:
# Draw a epitrochoid
def epitrochoid(R=10, r=7, d=6):
    # ==== RANGE AND FINENESS OF PARAMETER Θ ====
    # Find the necessary angle θ_max to return to close the curve:
    # I base the algorithm for finding the θ_max on following assumptions:
    # - Let O be the perimeter of the greater circle G and o that of the smaller circle S and let ν be
    # the radius vector of the center of the circle S being positioned in the center of the circle G
    # and θ the angle of ν.
    # Then the epitrochoid is closed (first time) if θ is such that $ n * O = m * o $
    # where n,m are natural numbers and $ θ = 2πn $ (n,m being the smallest possible).
    # The number n actually represents the number of revolutions of the radius vector ν (which is determined by θ).
    # - All numbers in a computer are rational, indeed - so irrational (for which a solution does not always exist)
    # are excluded. Then the smallest solution of the equation $ n * O = m * o $ can be found this way:
    #   - Find a fractional form of O and o -> $ O = O_n/O_d $ and $ o = o_n/o_d $ respectively
    #   - Find the least common denominator (LCD) of those fractions (i.e. $lcd = lcm(O_d, o_d)$)
    #   - Find the numerators (O_N, o_N) respective to the LCD
    #   - Find their least common multiple (LCM)
    #   - Then $ n = LCM / O_n $
    # - To find the value of n with perimeters O and o, although more reasonable and clearer is this way, it is
    # computationally inefficient and unnecessary as O and o differ from known values of radii R and r respectively
    # by a factor 2π, and so the value of n can be calculated directly using the radii R and r instead of
    # the perimeters O and o.
    # - Knowing n, we can simply calculate θ_max from $ θ = 2πn $ (stated above)
    R_frac = frac(R).limit_denominator(1000)                # Fractional representation of R (within the precision)
    r_frac = frac(r).limit_denominator(1000)                # Fractional representation of r (within the precision)
    lcd = math.lcm(R_frac.denominator, r_frac.denominator)  # LCD of R and r
    R_numer = int(R * lcd)                                  # Numerator for R with denominator lcd
    r_numer = int(r * lcd)                                  # Numerator for r with denominator lcd
    numers_lcm = math.lcm(R_numer, r_numer)                 # LCM of numerators respective to lcd
    revols_of_G = numers_lcm / R_numer                      # Revolutions of the greater circle G to close the curve
    θ_min = 0                            # Angle of the radius vector ν at the "starting point" of the curve
    θ_max = 2 * np.pi * revols_of_G      # The least angle θ necessary to close the curve
    θ_fineness = int(100 * revols_of_G)  # Fineness set to 100 points for each revolution

    # ==== DATA GENERATION ====
    θ = np.linspace(θ_min, θ_max, θ_fineness)  # Vector of values of θ
    λ = R + r  # Temporary value (distance of the centers of the circles)
    κ = λ / r  # Temporary value (koefficient of θ)
    φ = κ * θ  # Temporary variable for the second goniometrical function
    x = λ * np.cos(θ) - d * np.cos(φ)  # x-coordinate values for parameter θ
    y = λ * np.sin(θ) - d * np.sin(φ)  # y-coordinate values for parameter θ

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(8, 8), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.plot(x, y, color="blue", linewidth=1, linestyle='-')
    # Set title
    axes.set_title(f"Epitrochoid for ${R = }, {r = }, {d = }$")
    # Show the figure
    plt.show()

In [ ]:
# Tests
epitrochoid(R=10, r=7, d=1)
epitrochoid()
epitrochoid(R=10, r=7, d=8)
epitrochoid(R=2.1, r=2, d=1)

### "Polygonal hypotrochoid" (a curve described by a point fixed relatively to the circle rolled on n-gon from within)
The algorithm below is briefly described in the code comments. \
 \
**NOTE:** \
There is a parameter `limit` which serves to end up the algorithm untimely if the curve does not close in such a definite number of revolutions around the polygon. The reason for such approach is this: If a is a rational number and b is irrational one, then there is no "least common multiple" of those values (i.e. there are no natural coeficients n,m for the equation $ n * O = m * o $). This means that the curve would never close, i.e. there is no period for which it would evolve the same way for ever. Although there exist the coefficients m,n, they can be too large (even for a computer), so there are a lot of cases for which it practically never ends. (Besides, for rational lengths of sides of a polygon, the perimeter is rational; but if the radius of the moving circle is rational too, then its perimeter is rational*π which is irrational... It is represented in computers approximately as rational though; however, it leads to too large coefficients!)


In [17]:
# Draw a "polygonal hypotrochoid"
# a...length of a side of the polygon to be rolled from within
# n...number of angles/sides of polygon (n-gon)
# r...radius of the moving circle
# d...distance of the "drawing point" from the center of the moving circle
def polygonal_hypotrochoid(a=10, n=7, r=1, d=1.5, limit=30):
    # ==== TEST INPUT ====
    # Test inputs not suitable for polygons
    if not isinstance(n, int) or n < 1:
        raise ValueError("InvalidInput-CannotRepresentPolygon")

    # ==== INNER POLYGON PROPERTIES ====
    φ = 2 * np.pi / n                # The angle at the vertex of the triangle which lies at the center of n-gon
    R_o = a / (2 * math.tan(φ / 2))  # The radius of the circle inscribed in the outer n-gon
    R_i = R_o - r                    # The radius of the circle inscribed in the inner n-gon
    a_i = 2 * R_i * math.tan(φ / 2)  # The length of the side of the inner n-gon
    # Test invalid radius of the moving circle
    if not R_i > r:
        raise ValueError("InvalidInput-MovingCircleDoesNotFitInPolygon")

    # ==== REVOLUTIONS COUNT & DETAIL SPECIFICATION (discretization parameters) ====
    # Find the necessary number of revolutions of the moving circle around the polygon to close the curve
    # (same as for the circles except that the perimeters should be used
    # - no radii can be used)
    O = n * a_i        # Perimeter of the inner n-gon (corresponds with the line segments rolled on the outer n-gon)
    o = 2 * np.pi * r  # Perimeter of the moving circle
    O_frac = frac(O).limit_denominator(1000)                # Fractional representation of O (within the precision)
    o_frac = frac(o).limit_denominator(1000)                # Fractional representation of o (within the precision)
    lcd = math.lcm(O_frac.denominator, o_frac.denominator)  # LCD of O and o
    O_numer = int(O * lcd)                                  # Numerator for O with denominator lcd
    o_numer = int(o * lcd)                                  # Numerator for o with denominator lcd
    numers_lcm = math.lcm(O_numer, o_numer)                 # LCM of numerators respective to lcd
    # Revolutions of the moving circle around the n-gon to close the curve
    revols_around_ngon = int(numers_lcm / O_numer)
    # For the case of numbers with large lcm limit the number of revolutions
    if limit < revols_around_ngon:
        # Revolutions of the moving circle around the n-gon to "nearly close" the curve
        revols_around_ngon = limit
    total_len = revols_around_ngon * n  # The total number of sides to pass to close the curve
    detail = 1000 + 1                   # Sets how many points would be calculated for one side

    # ==== INTERVAL DIVISION (discretization) ====
    # Division of the interval of periodicity of the curve
    interval_div = np.linspace(0, total_len * a_i, total_len * detail +
                               1)[0:-1]  # The last being excluded as superfluous

    # ==== INIT. STRAIGHT LINE - COMPLEX COORDINATES ====
    # Complex coordinates of the center of the moving circle (cmc) with respect to the center of the polygon (cp)
    # Translation along the y-axis down by a_i / 2 (to let the first/base side be positioned in its proper place)
    X_cmccp = R_i + 1j * (interval_div - a_i / 2)

    # ==== CIRCULAR MOVEMENT AROUND MOVING CIRCLE - COMPLEX COORDINATES ====
    # ---- Generating angles for a cycloid along a straight line (preparations for transformations) ----
    # Calculating the angles by which the moving circle rotated during rolling along the vertical straight line
    θ = - interval_div / r
    # Correcting phase shift caused by a breaking of the straight line (it is not a cycloid now)
    for i in range(1, total_len):
        θ[(i * detail):] += -φ  # Minus sign is derived from the negative angle of rotation (around cmc)
    # ---- Generating cartesian coordinates in a complex notation ----
    # Complex coordinates of a point on the curve (pc) with respect to the center of the moving circle (cmc)
    X_pccmc = d * (np.cos(θ) + 1j * np.sin(θ))  # Parametric equations of a circle (of the moving circle)

    # ==== COMPOSITE MOVEMENT - COMPLEX COORDINATES ====
    # Complex coordinates of a point on the curve (pc) with respect to the center of the polygon (cp)
    X_pccp = X_pccmc + X_cmccp  # Composition of the relative coordinates and the "absolute" coordinates

    # ==== FIRST TRANSFORMATION - TRANSLATION PER PARTES - COMPLEX COORDINATES ====
    # Translate all segments down to the base position of the first segment
    for i in range(1, total_len):
        # Translate all segments from the i-th to the last down along y-axis by a_i (the length of one segment)
        X_pccp[(i * detail):] -= 1j * a_i

    # ==== SECOND TRANSFORMATION - ROTATION PER PARTES - COMPLEX COORDINATES ====
    # Rotation "vector" (a unit complex number with angle φ - this makes
    # counterclockwise rotation by φ of any complex number it multiplies)
    e_φ = math.cos(φ) + 1j * math.sin(φ)  # IMPORTANT POINT for the transformation!
    # Rotate all (except the first) segments counterclockwise by φ, then
    # rotate all except the first two again, then
    # rotate all except the first tree again, then... 
    # (in the end all segments should be at their proper positions)
    for i in range(1, total_len):
        X_pccp[(i * detail):] *= e_φ  # Rotate all segments from the i-th to the last by φ (counterclockwise)

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(10, 10), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.plot(X_pccp.real, X_pccp.imag, color="blue", linewidth=1, linestyle='-')
    # Set title
    axes.set_title(f"Polygonal hypotrochoid for ${a = }, {n = }, {r = }, {d = }, {limit = }$")
    # Show the figure
    plt.show()

In [ ]:
# Tests
polygonal_hypotrochoid()
polygonal_hypotrochoid(d=1.5, limit=20)
polygonal_hypotrochoid(a=5, n=6, r=2, limit=100)
polygonal_hypotrochoid(a=200, n=3, r=5, d=20, limit=2)
polygonal_hypotrochoid(a=200, n=4, r=5, d=20, limit=2)
polygonal_hypotrochoid(a=200, r=2, d=50, limit=1)
polygonal_hypotrochoid(a=200, n=6, r=2, d=50, limit=1)
polygonal_hypotrochoid(a=50, n=20, r=4, d=6, limit=2)   # Almost circle
polygonal_hypotrochoid(a=1, n=1000, r=4, d=6, limit=2)  # Almost perfect circle